# References

This notebook contains analysis of certificate references in Common Criteria certificates.

The notebook has two parts, an analysis part and a network visualization part.
But first some common initialization and data loading.

In [ ]:
from __future__ import annotations
import networkx as nx
import networkx.algorithms.community as nx_comm
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sec_certs.dataset.cc import CCDataset
import pandas as pd
import seaborn as sns
import numpy as np
import sys
from tqdm import tqdm
from pathlib import Path


sys.path.insert(0, "./../")
from fixed_sankey_plot import sankey

%matplotlib inline

# matplotlib.use("pgf")
sns.set_theme(style='white')
plt.rcParams["axes.linewidth"] = 0.5
plt.rcParams["legend.fontsize"] = 6.5
plt.rcParams["xtick.labelsize"] = 8
plt.rcParams["ytick.labelsize"] = 8
plt.rcParams["ytick.left"] = True
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 0.5
plt.rcParams['ytick.major.pad'] = 0
plt.rcParams["xtick.bottom"] = True
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 0.5
plt.rcParams['xtick.major.pad'] = 0
plt.rcParams["pgf.texsystem"] = "pdflatex"
plt.rcParams["font.family"] = "serif"
plt.rcParams["text.usetex"] = True
plt.rcParams["pgf.rcfonts"] = False
plt.rcParams["axes.titlesize"] = 8
plt.rcParams["legend.handletextpad"] = 0.3
plt.rcParams['lines.markersize'] = 4
plt.rcParams['savefig.pad_inches'] = 0.01
sns.set_palette("deep")

#plt.style.use("seaborn-whitegrid")
#sns.set_palette("deep")
#sns.set_context("notebook")  # Set to "paper" for use in paper :)

#plt.rcParams['figure.figsize'] = (10, 6)

RESULTS_DIR = Path("./results/references")
RESULTS_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
# Initialize
dset = CCDataset.from_web_latest()

In [ ]:
df = dset.to_pandas()
df_id_rich = df.loc[df.cert_id.notnull()].copy()

## Reference analysis


### Count numbers of reference-rich certificates

- From the numbers follows that whenever a certificate is directly referencing some else, it also indirectly references some else
- We have more outgoing references than ingoing references, which kinda makes sense. You don't have to be aware that some other cert references you

In [ ]:
df["has_outgoing_direct_references"] = df.directly_referencing.notnull()
df["has_incoming_direct_references"] = df.directly_referenced_by.notnull()
df["has_outgoing_indirect_references"] = df.indirectly_referencing.notnull()
df["has_incoming_indirect_references"] = df.indirectly_referenced_by.notnull()

df_id_rich["has_outgoing_direct_references"] = df_id_rich.directly_referencing.notnull()
df_id_rich["has_incoming_direct_references"] = df_id_rich.directly_referenced_by.notnull()
df_id_rich["has_outgoing_indirect_references"] = df_id_rich.indirectly_referencing.notnull()
df_id_rich["has_incoming_indirect_references"] = df_id_rich.indirectly_referenced_by.notnull()

archived_cert_id_list = set(df_id_rich[df_id_rich.status == "archived"].cert_id)

In [ ]:
def len_if_exists(x) -> int:
    return len(x) if pd.notnull(x) else 0

df_id_rich["n_references"] = df_id_rich.directly_referencing.map(len_if_exists)
df_id_rich["n_trans_references"] = df_id_rich.indirectly_referencing.map(len_if_exists)
df_id_rich["n_referenced_by"] = df_id_rich.directly_referenced_by.map(len_if_exists)
df_id_rich["n_trans_referenced_by"] = df_id_rich.indirectly_referenced_by.map(len_if_exists)

n_ref_smartcards = df_id_rich.loc[(df_id_rich.directly_referencing.notnull()) & (df_id_rich.category == "ICs, Smart Cards and Smart Card-Related Devices and Systems")].shape[0]
n_ref_others = df_id_rich.loc[(df_id_rich.directly_referencing.notnull()) & (df_id_rich.category != "ICs, Smart Cards and Smart Card-Related Devices and Systems")].shape[0]
print(f"Number of smartcard certificates that reference some other certificate: {n_ref_smartcards}")
print(f"Number of non-smartcard certificates that reference some other certificate: {n_ref_others}")

## Distribution of references and certificate reach

In [ ]:
df_melted = df_id_rich[["n_references", "n_trans_references", "n_referenced_by", "n_trans_referenced_by"]].melt()
df_melted["incoming"] = df_melted.variable.map(lambda x: False if x.endswith("by") else True)
sns.catplot(data=df_melted, kind="violin", x="variable", y="value", col="incoming", sharex=False, sharey=False)
plt.savefig(RESULTS_DIR / "violin_n_references.pdf", bbox_inches="tight")
plt.show()

## Evolution of certificate reach for top-10 certificates

In [ ]:
top_10 = df_id_rich.n_trans_referenced_by.sort_values(ascending=False).head(10)
top_10_cert_ids = df_id_rich.loc[top_10.index].cert_id
top_10

In [ ]:
# TODO: Check if the computation can be vectorized through some time-series or other thing...
def find_reach_over_time(df: pd.DataFrame, cert_id: str, date_range: pd.DatetimeIndex) -> pd.Series:
    df_copy = df.copy()
    df_copy["is_referenced_by"] = df_copy.indirectly_referencing.map(lambda x: False if pd.isnull(x) else cert_id in x)
    df_subset = df_copy.loc[df_copy.is_referenced_by]
    dct = {}
    for date in date_range:
        dct[date] = df_subset.loc[(date >= df_subset.not_valid_before) & (date < df_subset.not_valid_after)].shape[0]
    return pd.Series(dct, name=cert_id)

certs_with_top_reach = top_10_cert_ids.tolist()
date_range = pd.date_range(df_id_rich.not_valid_before.min(), df_id_rich.not_valid_before.max())
data = [find_reach_over_time(df_id_rich, x, date_range) for x in tqdm(certs_with_top_reach)]
df_reach_evolution = pd.concat(data, axis=1)

df_reach_evolution.index.name = "date"
df_reach_evolution = df_reach_evolution.reset_index()


df_reach_evolution_melted = df_reach_evolution.melt(id_vars="date", var_name="certificate", value_name="reach")

g = sns.lineplot(data=df_reach_evolution_melted, x="date", y="reach", hue="certificate")
g.set(title="Certificate reach over time", xlabel="Time", ylabel="Certificate reach")
plt.savefig(RESULTS_DIR / "lineplot_top_certificate_reach.pdf", bbox_inches="tight")
plt.show()

## Average number of references & certificate reach over time

In [ ]:
def compute_avg_references(df: pd.DataFrame, smartcards: bool, variable: str) -> pd.DataFrame:
    """
    Computes a series where index is date and value is average 'variable' value for certificates that were valid at that time,
    coming from either smardcards or other categories. E.g., variable can be `directly_referencing`.
    """
    df_copy = df.copy()
    df_copy["target_variable"] = df[variable].map(lambda x: len(x) if pd.notnull(x) else 0)

    date_range = pd.date_range(df_copy.not_valid_before.min(), df_copy.not_valid_before.max())
    dct = {}
    for date in date_range:
        if smartcards:
            dct[date] = df_copy.loc[(date >= df_copy.not_valid_before) & (date < df_copy.not_valid_after) & (df_copy.category == "ICs, Smart Cards and Smart Card-Related Devices and Systems")]["target_variable"].mean()
        else:
            dct[date] = df_copy.loc[(date >= df_copy.not_valid_before) & (date < df_copy.not_valid_after) & (df_copy.category != "ICs, Smart Cards and Smart Card-Related Devices and Systems")]["target_variable"].mean()
    series_name = variable + " smartcards" if smartcards else variable + " other categories"
    return pd.Series(dct, name=series_name)   

refs_smartcards = compute_avg_references(df_id_rich, True, "directly_referencing")
trans_refs_smartcards = compute_avg_references(df_id_rich, True, "indirectly_referencing")
refs_others = compute_avg_references(df_id_rich, False, "directly_referencing")
trans_refs_others = compute_avg_references(df_id_rich, False, "indirectly_referencing")
df_avg_num_refs = pd.concat([refs_smartcards, refs_others, trans_refs_smartcards, trans_refs_others], axis=1)
df_avg_num_refs.index.name = "date"
df_avg_num_refs = df_avg_num_refs.reset_index()
df_avg_num_refs_melted = df_avg_num_refs.melt(id_vars=["date"], var_name="category", value_name="n_references")

reach_smartcards = compute_avg_references(df_id_rich, True, "indirectly_referenced_by")
reach_others = compute_avg_references(df_id_rich, False, "indirectly_referenced_by")
df_avg_reach = pd.concat([reach_smartcards, reach_others], axis=1)
df_avg_reach.index.name = "date"
df_avg_reach = df_avg_reach.reset_index()
df_avg_reach_melted = df_avg_reach.melt(id_vars=["date"], var_name="category", value_name="certificate reach")

g = sns.lineplot(data=df_avg_num_refs_melted, x="date", y="n_references", hue="category")
g.set(title="Average number of references in certificates", xlabel="Time", ylabel="Number of references")
plt.savefig(RESULTS_DIR / "lineplot_avg_n_references.pdf", bbox_inches="tight")
plt.show()

g = sns.lineplot(data=df_avg_reach_melted, x="date", y="certificate reach", hue="category")
g.set(title="Average reach of a certificate in time", xlabel="Time", ylabel="Reach")
plt.savefig(RESULTS_DIR / "lineplot_avg_reach.pdf", bbox_inches="tight")
plt.show()

### Plot direct references per category (count plot)

In [ ]:
figure, axes = plt.subplots(1, 2)
figure.set_size_inches(16, 10)
figure.set_tight_layout(True)

col_to_depict = ["has_outgoing_direct_references", "has_incoming_direct_references"]

for index, col in enumerate(col_to_depict):
    countplot = sns.countplot(data=df, x="category", hue=col, ax=axes[index])
    countplot.set(
        xlabel="Category",
        ylabel="Outgoing direct references",
        title=f"Countplot of {' '.join(col.split('_'))}",
    )
    countplot.tick_params(axis="x", rotation=90)
    countplot.legend(title=' '.join(col.split('_')), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.savefig(str(RESULTS_DIR / "references_countplot.pdf"), bbox_inches="tight")
plt.close(figure)


### Plot direct references per category (Sankey diagram)

In [ ]:
cert_id_to_category_mapping = dict(zip(df.cert_id, df.category))
cert_id_to_category_mapping[np.NaN] = "No references"

exploded = df_id_rich.loc[:, ["category", "directly_referencing"]].explode("directly_referencing")
exploded["ref_category"] = exploded.directly_referencing.map(lambda x: cert_id_to_category_mapping[x] if pd.notnull(x) else np.nan)
exploded = exploded.loc[exploded.ref_category.notnull()]

exploded_with_refs = exploded.loc[exploded.ref_category != "No references"]

all_categories = set(exploded.category.unique()) | set(exploded.ref_category.unique())
colors = list(sns.color_palette("hls", len(all_categories), as_cmap=False).as_hex())
color_dict = dict(zip(all_categories, colors))

figure, axes = plt.subplots(1, 1)
figure.set_size_inches(24, 10)
figure.set_tight_layout(True)

sankey(exploded.category, exploded.ref_category, colorDict=color_dict, leftLabels=list(exploded.category.unique()), rightLabels=list(exploded.ref_category.unique()), fontsize=12, ax=axes)

figure.savefig(str(RESULTS_DIR / "category_references.pdf"), bbox_inches="tight")
figure.savefig(str(RESULTS_DIR / "category_references.pgf"), bbox_inches="tight")
plt.close(figure)

### Plot direct references per scheme (count plot)

In [ ]:
figure, axes = plt.subplots(1, 2)
figure.set_size_inches(14, 4)
figure.set_tight_layout(True)

col_to_depict = ["has_outgoing_direct_references", "has_incoming_direct_references"]

for index, col in enumerate(col_to_depict):
    countplot = sns.countplot(data=df, x="scheme", hue=col, ax=axes[index])
    countplot.set(
        xlabel="Category",
        ylabel="Outgoing direct references",
        title=f"Countplot of {' '.join(col.split('_'))}",
    )
    countplot.tick_params(axis="x", rotation=90)
    countplot.legend(title=' '.join(col.split('_')), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.savefig(str(RESULTS_DIR / "references_per_scheme_countplot.pdf"), bbox_inches="tight")
plt.close(figure)

### Number of certificates referencing archived certificates (count plot)

In [ ]:
def references_archived_cert(references):
    if pd.isnull(references):
        return False

    return any([x in cert_ids] for x in references)

cert_ids = set(df.loc[((df.cert_id.notnull()) & (df.status == "archived")), "cert_id"].tolist())
df["references_archived_cert"] = df.directly_referenced_by.map(references_archived_cert)

# TODO: We should limit on the number of certificates that referenced an archived certificate at some point where they were active as well.
print(f"Number of certificates that reference some archived certificate: {df.loc[df.references_archived_cert].shape[0]}")

col_to_depict = ["category", "scheme"]

figure, axes = plt.subplots(1, 2)
figure.set_size_inches(14, 8)
figure.set_tight_layout(True)

for index, col in enumerate(col_to_depict):
    countplot = sns.countplot(data=df, x=col, hue="references_archived_cert", ax=axes[index])
    countplot.set(
        xlabel=col,
        ylabel="Outgoing direct references",
        title="Countplot of certificates that reference some archived certificate",
    )
    countplot.tick_params(axis="x", rotation=90)
    countplot.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

### Count scheme references (Sankey diagram)

In [ ]:
cert_id_to_scheme_mapping = dict(zip(df.cert_id, df.scheme))

df_ref_rich = df_id_rich.loc[df.directly_referencing.notnull()]
exploded = df_ref_rich.loc[:, ["scheme", "directly_referencing"]].explode("directly_referencing")

exploded["ref_scheme"] = exploded.directly_referencing.map(cert_id_to_scheme_mapping)
exploded = exploded.loc[exploded.ref_scheme.notnull()]

all_schemes = set(exploded.scheme.unique()) | set(exploded.ref_scheme.unique())
colors = list(sns.color_palette("hls", len(all_schemes), as_cmap=False).as_hex())
color_dict = dict(zip(all_schemes, colors))

figure, axes = plt.subplots(1, 1)
figure.set_size_inches(4, 4)
figure.set_tight_layout(True)

sankey(exploded.scheme, exploded.ref_scheme, colorDict=color_dict, leftLabels=list(exploded.scheme.unique()), rightLabels=list(exploded.ref_scheme.unique()), fontsize=7, ax=axes)

figure.savefig(str(RESULTS_DIR / "scheme_references.pdf"), bbox_inches="tight")
figure.savefig(str(RESULTS_DIR / "scheme_references.pgf"), bbox_inches="tight")
plt.close(figure)

### Temporal evolution of references

Shows plot with relative number of certificates for a given year that reference some other certificate

In [ ]:
# TODO: Again, this plot is neither shown nor saved as a figure
df_temporal = df.loc[df.year_from < 2022].groupby(["year_from"])["directly_referencing"].count().reset_index().set_index("year_from")
n_issued_certs = df.groupby("year_from").name.count().reset_index().rename(columns={"name": "n_certs"}).set_index("year_from")
df_temporal.directly_referencing = 100 * df_temporal.directly_referencing / n_issued_certs.n_certs

line = sns.lineplot(data=df_temporal, x="year_from", y="directly_referencing")
line.yaxis.set_major_formatter(mtick.PercentFormatter())

### Cross references

## Reference network visualization

In [ ]:
certs_with_ids = {cert.heuristics.cert_id: cert for cert in dset if cert.heuristics.cert_id}

print(f"Certificates in dataset: {len(dset)}")
print(f"Certificates with extracted IDs: {len(certs_with_ids)}")

### Certificate report references

In [ ]:
refs_cr = nx.DiGraph()
for cert_id, cert in certs_with_ids.items():
    refs_cr.add_node(cert_id, cert=cert)
for cert_id, cert in certs_with_ids.items():
    if cr_refs := cert.heuristics.report_references.directly_referencing:
        for ref_id in cr_refs:
            if ref_id in certs_with_ids:
                refs_cr.add_edge(cert_id, ref_id, type=("cr",))
print(f"References in certificate reports: {len(refs_cr.edges)}")

### Security target references

In [ ]:
refs_st = nx.DiGraph()
for cert_id, cert in certs_with_ids.items():
    refs_st.add_node(cert_id, cert=cert)
for cert_id, cert in certs_with_ids.items():
    if st_refs := cert.heuristics.st_references.directly_referencing:
        for ref_id in st_refs:
            if ref_id in certs_with_ids:
                refs_st.add_edge(cert_id, ref_id, type=("st",))
print(f"References in security targets: {len(refs_st.edges)}")

### Combined references

In [ ]:
refs = nx.DiGraph()
for cert_id, cert in certs_with_ids.items():
    refs.add_node(cert_id, cert=cert)

for cert_id, cert in certs_with_ids.items():
    cr_refs = cert.heuristics.report_references.directly_referencing
    st_refs = cert.heuristics.st_references.directly_referencing
    cr_refs = set(cr_refs) if cr_refs is not None else set()
    st_refs = set(st_refs) if st_refs is not None else set()
    both = cr_refs.union(st_refs)
    for ref in both:
        if ref not in certs_with_ids:
            continue
        if ref in cr_refs and ref not in st_refs:
            refs.add_edge(cert_id, ref, type=("cr", ))
        elif ref in st_refs and ref not in cr_refs:
            refs.add_edge(cert_id, ref, type=("st", ))
        else:
            refs.add_edge(cert_id, ref, type=("cr", "st"))
print(f"Combined references (not double counted): {len(refs.edges)}")

### Certificate overview
Enter the certificate you are interested in below and see its reference graph component.

In [ ]:
cert_id = "ANSSI-CC-2019/02"

In [ ]:
cert = certs_with_ids.get(cert_id)
if cert is None:
    print(f"Certificate with id {cert_id} is not present in the dataset.")

for component in nx.weakly_connected_components(refs):
    if cert_id in component:
        break

view = nx.subgraph_view(refs, lambda node: node in component)
print(f"Certificate with id {cert_id}:")
print(f" - is in a component with {len(view.nodes)} certificates and {len(view.edges)} references.")
print(f" - references {list(view[cert_id].keys())}")
print(f" - is referenced by {list(view.predecessors(cert_id))}")
print(f" - its page is at https://seccerts.org/cc/{cert.dgst}/")

In [ ]:
# TODO: Not sure what this thing does
nx.draw(view, pos=nx.planar_layout(view), with_labels=True)

## Some graph metrics
From <https://dataground.io/2021/09/29/simple-graph-metrics-networkx-for-beginners/> and
<https://theslaps.medium.com/centrality-metrics-via-networkx-python-e13e60ba2740>.
Also good <https://www.geeksforgeeks.org/network-centrality-measures-in-a-graph-using-networkx-python/>

In [ ]:
print(f"Density = {nx.density(refs)}")
print(f"Transitivity = {nx.transitivity(refs)}")

In [ ]:
print("Degree centrality <Popularity> (top 20):")
degree_centrality_vals = [(node, val) for node, val in nx.degree_centrality(refs).items()]
degree_centrality_vals.sort(key=lambda pair: pair[1], reverse=True)
for pair in degree_centrality_vals[:20]:
    print(f"\t{pair[0]} = {pair[1]}")

In [ ]:
print("Eigenvector centrality <Influence> (top 20):")
eigenvector_centrality_vals = [(node, val) for node, val in nx.eigenvector_centrality(refs).items()]
eigenvector_centrality_vals.sort(key=lambda pair: pair[1], reverse=True)
for pair in eigenvector_centrality_vals[:20]:
    print(f"\t{pair[0]} = {pair[1]}")

In [ ]:
print("Closeness centrality <Centralness> (top 20):")
closeness_centrality_vals = [(node, val) for node, val in nx.closeness_centrality(refs).items()]
closeness_centrality_vals.sort(key=lambda pair: pair[1], reverse=True)
for pair in closeness_centrality_vals[:20]:
    print(f"\t{pair[0]} = {pair[1]}")

In [ ]:
print("Betweenness centrality <Bridge> (top 20):")
betweenness_centrality_vals = [(node, val) for node, val in nx.betweenness_centrality(refs).items()]
betweenness_centrality_vals.sort(key=lambda pair: pair[1], reverse=True)
for pair in betweenness_centrality_vals[:20]:
    print(f"\t{pair[0]} = {pair[1]}")

In [ ]:
component_lengths = list(filter(lambda comp_len: comp_len > 1, map(len, nx.weakly_connected_components(refs))))
component_lengths.sort(reverse=True)
print(component_lengths)

In [ ]:
big_boy = refs.subgraph(max(nx.weakly_connected_components(refs), key=len))
communities = list(nx_comm.greedy_modularity_communities(big_boy))
print(len(communities))

In [ ]:
for com in communities:
    for i in sorted(com):
        print(f"\t{i}")

## LaTeX commands

In [ ]:
def contains_archived_cert_reference(referencing):
    if pd.isnull(referencing):
        return False
    return bool(archived_cert_id_list.intersection(referencing))


print(f"\\newcommand{{\\numCcAllDirectReferencing}}{{{df.has_outgoing_direct_references.sum()}}}")
print(f"\\newcommand{{\\numCcAllNotDirectReferencing}}{{{len(df) - df.has_outgoing_direct_references.sum()}}}")
print(f"\\newcommand{{\\numCcWithIdDirectReferencing}}{{{df_id_rich.has_outgoing_direct_references.sum()}}}")
print(f"\\newcommand{{\\numCcWithIdNotDirectReferencing}}{{{len(df_id_rich) - df_id_rich.has_outgoing_direct_references.sum()}}}")
print(f"\\newcommand{{\\numCCActiveDirectReferencing}}{{{df_id_rich.loc[df_id_rich.status == 'active'].has_outgoing_direct_references.sum()}}}")
print(f"\\newcommand{{\\numCCActiveDirectReferencingArchived}}{{{df_id_rich[df_id_rich.status == 'active'].directly_referencing.apply(contains_archived_cert_reference).sum()}}}")

print("")
print(f"\\newcommand{{\\numCCDirectRefsSameCategory}}{{{(exploded_with_refs.category == exploded_with_refs.ref_category).sum()}}}")
print(f"\\newcommand{{\\numCCDirectRefsOtherCategory}}{{{(exploded_with_refs.category != exploded_with_refs.ref_category).sum()}}}")
print(f"\\newcommand{{\\numCCDirectRefs}}{{{len(exploded_with_refs)}}}")
print(f"\\newcommand{{\\numCCDirectRefsFromSmartcards}}{{{(exploded_with_refs.category == 'ICs, Smart Cards and Smart Card-Related Devices and Systems').sum()}}}")

print("")
print(f"\\newcommand{{\\numCCUSReferencing}}{{{len(df_id_rich.loc[(df_id_rich.scheme == 'US') & (df_id_rich.directly_referencing.notnull())])}}}")
print(f"\\newcommand{{\\numCCUS}}{{{len(df_id_rich.loc[(df_id_rich.scheme == 'US')])}}}")